## Close up on DOSE

In [1]:
import pydeck as pdk
import ibis as ib
from ibis import _
import pandas as pd
import geopandas as gpd
import pydeck as pdk
import shapely
import numpy as np
import json
import io

ib.options.interactive = True
import pypalettes
import duckdb as ddb

In [2]:
# Method to plot polygons taken from: https://thedatasavvycorner.com/blogs/10-pydeck-with-gpd

# common paremeters to the deck maps
lat_center = 0
lon_center = 0

view_state = pdk.ViewState(latitude=lat_center
                           ,longitude=lon_center
                           ,zoom=1
                           ,bearing=0
                           ,pitch=0)




## Data

From: https://www.nature.com/articles/s41597-023-02323-8

In [3]:
dose_gpkg = "datasets/DOSE/DOSE_shapefiles.gpkg"
gpd.list_layers(dose_gpkg)

DataSourceError: datasets/DOSE/DOSE_shapefiles.gpkg: No such file or directory

In [ ]:
dose = gpd.read_file(dose_gpkg)

/Users/cenv1069/micromamba/envs/global-data/lib/python3.12/site-packages/pyogrio/geopandas.py:261: UserWarning: More than one layer found in 'DOSE_shapefiles.gpkg': 'all_non_GADM_regions' (default), 'layer_styles'. Specify layer parameter to avoid this warning.
  result = read_func(


In [ ]:
dose_csv = ib.read_csv("datasets/DOSE/DOSE_V2.csv",table_name="dose",ignore_errors=True)

In [ ]:
print(dose.shape)
dose.head()

(73, 11)


,GID_0,NAME_0,GID_1,NAME_1,VARNAME_1,NL_NAME_1,TYPE_1,ENGTYPE_1,CC_1,HASC_1,geometry
0,PHL,Philippines,PHL.110_1,Cordillera Administrative Region,None,None,None,None,None,None,"MULTIPOLYGON Z (((121.22208 18.50058 0, 121.22..."
1,PHL,Philippines,PHL.111_1,National Capital Region,None,None,None,None,None,None,"MULTIPOLYGON Z (((121.03842 14.78525 0, 121.03..."
2,PHL,Philippines,PHL.112_1,Ilocos Region,None,None,None,None,None,None,"MULTIPOLYGON Z (((119.86596 15.81539 0, 119.86..."
3,PHL,Philippines,PHL.113_1,Cagayan Valley,None,None,None,None,None,None,"MULTIPOLYGON Z (((122.46667 16.92135 0, 122.46..."
4,PHL,Philippines,PHL.114_1,Central Luzon,None,None,None,None,None,None,"MULTIPOLYGON Z (((120.11687 14.76309 0, 120.11..."


In [ ]:
dose_csv.head()

┏━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ country ┃ region ┃ GID_0  ┃ GID_1   ┃ year  ┃ grp_lcu      ┃ pop    ┃ grp_pc_lcu ┃ ag_grp_pc_lcu ┃ man_grp_pc_lcu ┃ serv_grp_pc_lcu ┃ grp_pc_usd  ┃ ag_grp_pc_usd ┃ man_grp_pc_usd ┃ serv_grp_pc_usd ┃ grp_pc_lcu_2015 ┃ ag_grp_pc_lcu_2015 ┃ man_grp_pc_lcu_2015 ┃ serv_grp_pc_lcu_2015 ┃ grp_pc_usd_2015 ┃ ag_grp_pc_usd_2015 ┃ man_grp_pc_usd_2015 ┃ serv_grp_pc_usd_2015 ┃ grp_pc_lcu2015_usd ┃ ag_grp_pc_lcu2015_usd ┃ man_grp_pc_lcu2015_usd ┃ serv_grp_pc_lcu2015_usd ┃ cpi_2015   ┃ deflator_2015 ┃ fx         ┃ PPP       ┃ StructChange ┃ version ┃ T_a       ┃ P_a         ┃
┡━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ string  │ string │ string │ string  │ int64 │ float64      │ int64  │ float64    │ float64       │ float64        │ float64         │ float64     │ float64       │ float64        │ float64         │ float64         │ float64            │ float64             │ float64              │ float64         │ float64            │ float64             │ float64              │ float64            │ float64               │ float64                │ float64                 │ float64    │ float64       │ float64    │ float64   │ int64        │ float64 │ float64   │ float64     │
├─────────┼────────┼────────┼─────────┼───────┼──────────────┼────────┼────────────┼───────────────┼────────────────┼─────────────────┼─────────────┼───────────────┼────────────────┼─────────────────┼─────────────────┼────────────────────┼─────────────────────┼──────────────────────┼─────────────────┼────────────────────┼─────────────────────┼──────────────────────┼────────────────────┼───────────────────────┼────────────────────────┼─────────────────────────┼────────────┼───────────────┼────────────┼───────────┼──────────────┼─────────┼───────────┼─────────────┤
│ Albania │ Berat  │ ALB    │ ALB.1_1 │  2010 │ 4.941607e+10 │ 151375 │   326448.0 │          NULL │           NULL │            NULL │ 3140.842514 │          NULL │           NULL │            NULL │     345645.9835 │               NULL │                NULL │                 NULL │     3419.068968 │               NULL │                NULL │                 NULL │        2744.075141 │                  NULL │                   NULL │                    NULL │ 104.742951 │     94.445767 │ 103.936443 │ 44.196214 │            0 │     2.0 │ 13.123768 │ 1573.116224 │
│ Albania │ Berat  │ ALB    │ ALB.1_1 │  2011 │ 5.260391e+10 │ 148160 │   355048.0 │          NULL │           NULL │            NULL │ 3518.986662 │          NULL │           NULL │            NULL │     367423.0111 │               NULL │                NULL │                 NULL │     3752.327886 │               NULL │                NULL │                 NULL │        2916.962438 │                  NULL │                   NULL │                    NULL │ 113.207466 │     96.631944 │ 100.894955 │ 43.857685 │            0 │     2.0 │ 12.882040 │  796.287560 │
│ Albania 

In [ ]:
print(dose_csv.GID_0.nunique())
dose_csv.GID_0.value_counts()

┌────┐
│ 83 │
└────┘


┏━━━━━━━━┳━━━━━━━━━━━━━┓
┃ GID_0  ┃ GID_0_count ┃
┡━━━━━━━━╇━━━━━━━━━━━━━┩
│ string │ int64       │
├────────┼─────────────┤
│ PAN    │         232 │
│ SRB    │         175 │
│ EGY    │         290 │
│ ETH    │          21 │
│ GEO    │         142 │
│ GTM    │         105 │
│ JPN    │        2714 │
│ MKD    │         168 │
│ NPL    │          21 │
│ PAK    │         124 │
│ …      │           … │
└────────┴─────────────┘

In [ ]:
print(dose_csv.GID_1.nunique())
dose_csv.GID_1.value_counts()

┌──────┐
│ 1659 │
└──────┘


┏━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ GID_1    ┃ GID_1_count ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ string   │ int64       │
├──────────┼─────────────┤
│ NGA.6_1  │           5 │
│ NGA.16_1 │           5 │
│ NOR.19_1 │          28 │
│ PAK.5_1  │          31 │
│ PAN.4_1  │          25 │
│ PRY.10_1 │           5 │
│ PRY.16_1 │           5 │
│ PRY.15_1 │           5 │
│ PER.14_1 │          45 │
│ PER.23_1 │          45 │
│ …        │           … │
└──────────┴─────────────┘

### Adding the full GADM data set

In [ ]:
# connecting this table to ibis
conn = ib.duckdb.connect()

In [ ]:
# loading extensions
conn.raw_sql("""INSTALL spatial;LOAD spatial;""")

In [ ]:
conn.raw_sql("CREATE TABLE gadm AS SELECT * FROM ST_Read('datasets/boundaries/GADM/gadm_410.gpkg');")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [ ]:
# our table is there
conn.list_tables()

['gadm']

In [ ]:
gadm = conn.table("gadm")

In [ ]:
gadm.head()

┏━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ UID   ┃ GID_0  ┃ NAME_0      ┃ VARNAME_0 ┃ GID_1   ┃ NAME_1     ┃ VARNAME_1 ┃ NL_NAME_1 ┃ ISO_1  ┃ HASC_1 ┃ CC_1   ┃ TYPE_1  ┃ ENGTYPE_1 ┃ VALIDFR_1 ┃ GID_2     ┃ NAME_2    ┃ VARNAME_2 ┃ NL_NAME_2 ┃ HASC_2   ┃ CC_2   ┃ TYPE_2    ┃ ENGTYPE_2 ┃ VALIDFR_2 ┃ GID_3  ┃ NAME_3 ┃ VARNAME_3 ┃ NL_NAME_3 ┃ HASC_3 ┃ CC_3   ┃ TYPE_3 ┃ ENGTYPE_3 ┃ VALIDFR_3 ┃ GID_4  ┃ NAME_4 ┃ VARNAME_4 ┃ CC_4   ┃ TYPE_4 ┃ ENGTYPE_4 ┃ VALIDFR_4 ┃ GID_5  ┃ NAME_5 ┃ CC_5   ┃ TYPE_5 ┃ ENGTYPE_5 ┃ GOVERNEDBY ┃ SOVEREIGN   ┃ DISPUTEDBY ┃ REGION ┃ VARREGION ┃ COUNTRY     ┃ CONTINENT ┃ SUBCONT ┃ geom                                                                             ┃
┡━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ int64 │ string │ string      │ string    │ string  │ string     │ string    │ string    │ string │ string │ string │ string  │ string    │ string    │ string    │ string    │ string    │ string    │ string   │ string │ string    │ string    │ string    │ string │ string │ string    │ string    │ string │ string │ string │ string    │ string    │ string │ string │ string    │ string │ string │ string    │ string    │ string │ string │ string │ string │ string    │ string     │ string      │ string     │ string │ string    │ string      │ string    │ string  │ geospatial:geometry                                                              │
├───────┼────────┼─────────────┼───────────┼─────────┼────────────┼───────────┼───────────┼────────┼────────┼────────┼─────────┼───────────┼───────────┼───────────┼───────────┼───────────┼───────────┼──────────┼────────┼───────────┼───────────┼───────────┼────────┼────────┼───────────┼───────────┼────────┼────────┼────────┼───────────┼───────────┼────────┼────────┼───────────┼────────┼────────┼───────────┼───────────┼────────┼────────┼────────┼────────┼───────────┼────────────┼─────────────┼────────────┼────────┼───────────┼─────────────┼───────────┼─────────┼──────────────────────────────────────────────────────────────────────────────────┤
│     1 │ AFG    │ Afghanistan │ ~         │ AFG.1_1 │ Badakhshan │ Badahšan  │ ~         │ ~      │ AF.BD  │ ~      │ Velayat │ Province  │ Unknown   │ AFG.1.1_1 │ Baharak   │ ~         │ ~         │ AF.BD.BA │ ~      │ Wuleswali │ District  │ Unknown   │ ~      │ ~      │ ~         │ ~         │ ~      │ ~      │ ~      │ ~         │ ~         │ ~      │ ~      │ ~         │ ~      │ ~      │ ~         │ ~         │ ~      │ ~      │ ~      │ ~      │ ~         │ ~          │ Afghanistan │ ~          │ ~      │ ~         │ Afghanistan │ Asia      │ ~       │ <MULTIPOLYGON (((71.411 36.557, 71.41 36.552, 71.396 36.555, 71.374 36.555, ...> │
│     2 │ AFG    │ Afghanistan │ ~         │ AFG.1_1 │ Badakhshan │ Badahšan  │ ~         │ ~      │

YEAH !! connected to the table

In [ ]:
gadm.columns

['UID',
 'GID_0',
 'NAME_0',
 'VARNAME_0',
 'GID_1',
 'NAME_1',
 'VARNAME_1',
 'NL_NAME_1',
 'ISO_1',
 'HASC_1',
 'CC_1',
 'TYPE_1',
 'ENGTYPE_1',
 'VALIDFR_1',
 'GID_2',
 'NAME_2',
 'VARNAME_2',
 'NL_NAME_2',
 'HASC_2',
 'CC_2',
 'TYPE_2',
 'ENGTYPE_2',
 'VALIDFR_2',
 'GID_3',
 'NAME_3',
 'VARNAME_3',
 'NL_NAME_3',
 'HASC_3',
 'CC_3',
 'TYPE_3',
 'ENGTYPE_3',
 'VALIDFR_3',
 'GID_4',
 'NAME_4',
 'VARNAME_4',
 'CC_4',
 'TYPE_4',
 'ENGTYPE_4',
 'VALIDFR_4',
 'GID_5',
 'NAME_5',
 'CC_5',
 'TYPE_5',
 'ENGTYPE_5',
 'GOVERNEDBY',
 'SOVEREIGN',
 'DISPUTEDBY',
 'REGION',
 'VARREGION',
 'COUNTRY',
 'CONTINENT',
 'SUBCONT',
 'geom']

In [ ]:
columns = ['UID',
 'GID_0',
 'NAME_0',
 'VARNAME_0',
 'GID_1',
 'NAME_1',
 'VARNAME_1',
 'NL_NAME_1',
 'ISO_1',
 'HASC_1',
 'CC_1',
 'TYPE_1',
 'ENGTYPE_1',
 'VALIDFR_1',
 'GID_2',
 'NAME_2',
 'VARNAME_2',
 'NL_NAME_2',
 'HASC_2',
 'CC_2',
 'TYPE_2',
 'ENGTYPE_2',
 'VALIDFR_2']

query="""select """ + ", ".join(columns) + """, ST_AsText(ST_Simplify(geom,0.05)) as geom from gadm;"""
query

'select UID, GID_0, NAME_0, VARNAME_0, GID_1, NAME_1, VARNAME_1, NL_NAME_1, ISO_1, HASC_1, CC_1, TYPE_1, ENGTYPE_1, VALIDFR_1, GID_2, NAME_2, VARNAME_2, NL_NAME_2, HASC_2, CC_2, TYPE_2, ENGTYPE_2, VALIDFR_2, ST_AsText(ST_Simplify(geom,0.05)) as geom from gadm;'

In [ ]:
gadm_df = gadm.sql(query=query).to_pandas()

In [ ]:
gadm_df["geometry"]=gpd.GeoSeries.from_wkt(gadm_df.geom,crs="EPSG:4326")

gadm_df = gpd.GeoDataFrame(gadm_df)

In [ ]:
gadm_df.shape

(356508, 25)

In [ ]:
# gadm_df.plot()
gadm_df.columns

Index(['UID', 'GID_0', 'NAME_0', 'VARNAME_0', 'GID_1', 'NAME_1', 'VARNAME_1',
       'NL_NAME_1', 'ISO_1', 'HASC_1', 'CC_1', 'TYPE_1', 'ENGTYPE_1',
       'VALIDFR_1', 'GID_2', 'NAME_2', 'VARNAME_2', 'NL_NAME_2', 'HASC_2',
       'CC_2', 'TYPE_2', 'ENGTYPE_2', 'VALIDFR_2', 'geom', 'geometry'],
      dtype='object')

### Back to DOSE

In [ ]:
dose.columns

Index(['GID_0', 'NAME_0', 'GID_1', 'NAME_1', 'VARNAME_1', 'NL_NAME_1',
       'TYPE_1', 'ENGTYPE_1', 'CC_1', 'HASC_1', 'geometry'],
      dtype='object')

In [ ]:
# keeping only the columns that correspond 
gadm_df = gadm_df[dose.columns]

In [ ]:
dose = pd.concat([dose,gadm_df])

In [ ]:
print(dose.columns)
print(dose_csv.columns)

Index(['GID_0', 'NAME_0', 'GID_1', 'NAME_1', 'VARNAME_1', 'NL_NAME_1',
       'TYPE_1', 'ENGTYPE_1', 'CC_1', 'HASC_1', 'geometry'],
      dtype='object')
['country', 'region', 'GID_0', 'GID_1', 'year', 'grp_lcu', 'pop', 'grp_pc_lcu', 'ag_grp_pc_lcu', 'man_grp_pc_lcu', 'serv_grp_pc_lcu', 'grp_pc_usd', 'ag_grp_pc_usd', 'man_grp_pc_usd', 'serv_grp_pc_usd', 'grp_pc_lcu_2015', 'ag_grp_pc_lcu_2015', 'man_grp_pc_lcu_2015', 'serv_grp_pc_lcu_2015', 'grp_pc_usd_2015', 'ag_grp_pc_usd_2015', 'man_grp_pc_usd_2015', 'serv_grp_pc_usd_2015', 'grp_pc_lcu2015_usd', 'ag_grp_pc_lcu2015_usd', 'man_grp_pc_lcu2015_usd', 'serv_grp_pc_lcu2015_usd', 'cpi_2015', 'deflator_2015', 'fx', 'PPP', 'StructChange', 'version', 'T_a', 'P_a']


In [ ]:
type(dose_csv)

ibis.expr.types.relations.Table

In [ ]:
dose_csv_df_ = dose_csv.to_pandas()

In [ ]:
len(dose_csv_df_.GID_1.unique())

1659

In [ ]:
# filtering down the csv file to the year 2019
dose_csv_df = dose_csv_df_.loc[dose_csv_df_.year==2019]

before merging, dissolve the data set to be GID_1

In [ ]:
dose = dose.dissolve(by="GID_1")

In [ ]:
print(dose_csv_df.shape)
print(dose.shape)

(1037, 35)
(3721, 10)


In [ ]:
print(dose.head(3))

                                                   geometry GID_0  \
GID_1                                                               
          MULTIPOLYGON (((-169.79375 -83.17917, -171.910...   ATA   
?         POLYGON ((30.59167 50.41236, 30.75911 50.46781...   UKR   
AFG.10_1  POLYGON ((64.22197 33.18085, 63.99013 33.5127,...   AFG   

               NAME_0 NAME_1              VARNAME_1 NL_NAME_1   TYPE_1  \
GID_1                                                                    
           Antarctica                                                    
?             Ukraine      ?                      ?         ?        ?   
AFG.10_1  Afghanistan   Ghor  Gawr|Ghore|Ghour|Ghur            Velayat   

         ENGTYPE_1 CC_1 HASC_1  
GID_1                           
                                
?                ?           ?  
AFG.10_1  Province       AF.GR  


In [ ]:
print(dose_csv_df.head(3))

    country  region GID_0    GID_1  year       grp_lcu       pop   grp_pc_lcu  \
9   Albania   Berat   ALB  ALB.1_1  2019  5.811884e+10  123581.0  470289.4636   
19  Albania   Dibër   ALB  ALB.2_1  2019  5.096350e+10  117403.0  434090.2745   
29  Albania  Durrës   ALB  ALB.3_1  2019  1.720000e+11  290412.0  593212.6296   

    ag_grp_pc_lcu  man_grp_pc_lcu  ...  man_grp_pc_lcu2015_usd  \
9             NaN             NaN  ...                     NaN   
19            NaN             NaN  ...                     NaN   
29            NaN             NaN  ...                     NaN   

    serv_grp_pc_lcu2015_usd  cpi_2015  deflator_2015          fx        PPP  \
9                       NaN  135.2598      103.54323  109.850833  42.343073   
19                      NaN  135.2598      103.54323  109.850833  42.343073   
29                      NaN  135.2598      103.54323  109.850833  42.343073   

    StructChange  version  T_a  P_a  
9              0      2.0  NaN  NaN  
19             0 

In [ ]:
dose_gadm = pd.merge(left=dose,right=dose_csv_df,on="GID_1",sort=False,how="right",validate="one_to_one")

In [ ]:
dose_gadm.columns

Index(['GID_1', 'geometry', 'GID_0_x', 'NAME_0', 'NAME_1', 'VARNAME_1',
       'NL_NAME_1', 'TYPE_1', 'ENGTYPE_1', 'CC_1', 'HASC_1', 'country',
       'region', 'GID_0_y', 'year', 'grp_lcu', 'pop', 'grp_pc_lcu',
       'ag_grp_pc_lcu', 'man_grp_pc_lcu', 'serv_grp_pc_lcu', 'grp_pc_usd',
       'ag_grp_pc_usd', 'man_grp_pc_usd', 'serv_grp_pc_usd', 'grp_pc_lcu_2015',
       'ag_grp_pc_lcu_2015', 'man_grp_pc_lcu_2015', 'serv_grp_pc_lcu_2015',
       'grp_pc_usd_2015', 'ag_grp_pc_usd_2015', 'man_grp_pc_usd_2015',
       'serv_grp_pc_usd_2015', 'grp_pc_lcu2015_usd', 'ag_grp_pc_lcu2015_usd',
       'man_grp_pc_lcu2015_usd', 'serv_grp_pc_lcu2015_usd', 'cpi_2015',
       'deflator_2015', 'fx', 'PPP', 'StructChange', 'version', 'T_a', 'P_a'],
      dtype='object')

In [ ]:
dose_gadm.head(10)

,GID_1,geometry,GID_0_x,NAME_0,NAME_1,VARNAME_1,NL_NAME_1,TYPE_1,ENGTYPE_1,CC_1,...,man_grp_pc_lcu2015_usd,serv_grp_pc_lcu2015_usd,cpi_2015,deflator_2015,fx,PPP,StructChange,version,T_a,P_a
0,ALB.1_1,"MULTIPOLYGON (((19.80608 40.67479, 19.89524 40...",ALB,Albania,Berat,,,Qark,County,80001,...,NaN,NaN,135.2598,103.54323,109.850833,42.343073,0,2.0,NaN,NaN
1,ALB.2_1,"MULTIPOLYGON (((19.99836 41.49919, 20.20848 41...",ALB,Albania,Dibër,Dibra|Dibrë,,Qark,County,80005,...,NaN,NaN,135.2598,103.54323,109.850833,42.343073,0,2.0,NaN,NaN
2,ALB.3_1,"MULTIPOLYGON (((19.61146 41.27916, 19.51623 41...",ALB,Albania,Durrës,Durresi|Durrsi|Durazzo,,Qark,County,80009,...,NaN,NaN,135.2598,103.54323,109.850833,42.343073,0,2.0,NaN,NaN
3,ALB.4_1,"MULTIPOLYGON (((20.0151 40.86002, 20.13433 40....",ALB,Albania,Elbasan,,,Qark,County,80012,...,NaN,NaN,135.2598,103.54323,109.850833,42.343073,0,2.0,NaN,NaN
4,ALB.5_1,"MULTIPOLYGON (((19.45445 40.64239, 19.55273 40...",ALB,Albania,Fier,,,Qark,County,80017,...,NaN,NaN,135.2598,103.54323,109.850833,42.343073,0,2.0,NaN,NaN
5,ALB.6_1,"MULTIPOLYGON (((20.10228 40.06572, 19.93932 40...",ALB,Albania,Gjirokastër,Gjinokastër|Argirocastro|Argyroc,,Qark,County,80021,...,NaN,NaN,135.2598,103.54323,109.850833,42.343073,0,2.0,NaN,NaN
6,ALB.7_1,"MULTIPOLYGON (((20.50568 40.25413, 20.71323 40...",ALB,Albania,Korçë,Coriza|Corizza|Koritsa|Koritza|K,,Qark,County,80025,...,NaN,NaN,135.2598,103.54323,109.850833,42.343073,0,2.0,NaN,NaN
7,ALB.8_1,"MULTIPOLYGON (((20.21244 41.91004, 20.32672 41...",ALB,Albania,Kukës,Kosova|Kossovo,,Qark,County,80030,...,NaN,NaN,135.2598,103.54323,109.850833,42.343073,0,2.0,NaN,NaN
8,ALB.9_1,"MULTIPOLYGON (((19.66766 41.73865, 19.57802 41...",ALB,Albania,Lezhë,Alessio|Lezha,,Qark,County,80034,...,NaN,NaN,135.2598,103.54323,109.850833,42.343073,0,2.0,NaN,NaN
9,ALB.10_1,"MULTIPOLYGON (((19.34229 41.90724, 19.37991 41...",ALB,Albania,Shkodër,Escútari|Shkodra|Skodë|Scutari,,Qark,County,80038,...,NaN,NaN,135.2598,103.54323,109.850833,42.343073,0,2.0,NaN,NaN


In [ ]:
# dose_gadm.loc[~(dose_gadm.GID_0_x==dose_gadm.GID_0_y),["GID_1"]]
# print(dose.loc[dose.GID_0=="BES"].head(3))
# print(dose_csv.filter(_.GID_0=="ANT"))

In [ ]:
# removing the GID_0_y column
# dose_gadm.drop("GID_0_y",inplace=True)
dose_gadm.drop(columns=["GID_0_y"],inplace=True)

In [ ]:
dose_gadm.shape

(1037, 44)

In [ ]:
dose_gadm.head()

,GID_1,geometry,GID_0_x,NAME_0,NAME_1,VARNAME_1,NL_NAME_1,TYPE_1,ENGTYPE_1,CC_1,...,man_grp_pc_lcu2015_usd,serv_grp_pc_lcu2015_usd,cpi_2015,deflator_2015,fx,PPP,StructChange,version,T_a,P_a
0,ALB.1_1,"MULTIPOLYGON (((19.80608 40.67479, 19.89524 40...",ALB,Albania,Berat,,,Qark,County,80001,...,NaN,NaN,135.2598,103.54323,109.850833,42.343073,0,2.0,NaN,NaN
1,ALB.2_1,"MULTIPOLYGON (((19.99836 41.49919, 20.20848 41...",ALB,Albania,Dibër,Dibra|Dibrë,,Qark,County,80005,...,NaN,NaN,135.2598,103.54323,109.850833,42.343073,0,2.0,NaN,NaN
2,ALB.3_1,"MULTIPOLYGON (((19.61146 41.27916, 19.51623 41...",ALB,Albania,Durrës,Durresi|Durrsi|Durazzo,,Qark,County,80009,...,NaN,NaN,135.2598,103.54323,109.850833,42.343073,0,2.0,NaN,NaN
3,ALB.4_1,"MULTIPOLYGON (((20.0151 40.86002, 20.13433 40....",ALB,Albania,Elbasan,,,Qark,County,80012,...,NaN,NaN,135.2598,103.54323,109.850833,42.343073,0,2.0,NaN,NaN
4,ALB.5_1,"MULTIPOLYGON (((19.45445 40.64239, 19.55273 40...",ALB,Albania,Fier,,,Qark,County,80017,...,NaN,NaN,135.2598,103.54323,109.850833,42.343073,0,2.0,NaN,NaN


In [ ]:

layer_total_dose = pdk.Layer(
    "GeoJsonLayer",# "PolygonLayer",
    dose_gadm.loc[0:100],
    pickable=True,
    opacity=0.1,
    stroked=True,
    filled=True,
    extruded=False,
    auto_highlight=True,
    line_width=50,
    line_width_max_pixels=10,
    # get_polygon="coordinates",
    get_fill_color= [200,40,30],
    get_line_color=[0, 0, 0]
)

# Create mapdeck object
deck_map_dose = pdk.Deck(layers=[layer_total_dose]
                    ,initial_view_state=view_state
                    ,tooltip= {"text": """properties.GID_1"""}
                    ,height=800
                    ,map_style="road"
                    )

In [ ]:
deck_map_dose.to_html("deck_maps/deck_dose.html")